# Climate for each glacier

In [ ]:
# basic
from tqdm.notebook import tqdm
import pandas as pd
import os

# spatial
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

## 1. Baseline climate

In [ ]:
os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2020-12-31")

In [ ]:
# template and source files file
example    = xr.open_dataset("/home/rooda/Dropbox/Coding/OGGM/example_nc.nc")
dem        = xr.open_dataset("GIS South/dem_patagonia005.tif").rename({'y':'lat', 'x':'lon', 'band_data':'hgt'})
dem        = dem.sel(band=1, drop=True).drop("spatial_ref").fillna(0)

# PMET v1.1
rename_dic = {'latitude':'lat', 'longitude':'lon'}
pp_pmet    = xr.open_dataset("Data/Precipitation/PP_PMETsim_1980_2020_v10m.nc", chunks = "auto").sel(time = period)
t2m_pmet   = xr.open_dataset("Data/Temperature/Tavg_PMETsim_1980_2020_v10m.nc", chunks = "auto").sel(time = period)

In [ ]:
# actual prepro
regridder  = xe.Regridder(dem, pp_pmet, "bilinear")
dem        = regridder(dem)
t2m_pmet   = regridder(t2m_pmet) # this should be fixed in PMET v1.2

pmet   = xr.merge([pp_pmet,  t2m_pmet, dem]).rename({'pp':'prcp', 't2m':'temp'})
pmet = pmet.rename(rename_dic)

pmet.prcp.attrs['units']  = "mm"
pmet.temp.attrs['units']  = "degc"
pmet.hgt.attrs['units']   = "m"

# glacier mask (we dont need to downscale the complete area)
rgi = gpd.read_file("GIS South/Glaciers/RGI7_Hydro.shp")[["geometry"]]
rgi = rgi.buffer(0.20) 

mask_pmet = regionmask.mask_geopandas(rgi, pmet)   >= 0
pmet   = pmet.where(mask_pmet,     drop = True)

pmet.to_netcdf("/home/rooda/Hydro_results/PMETsim_historical_OGGM.nc")

## 2. Future climate

In [ ]:
os.chdir('/home/rooda/Hydro_results/')
period = slice("1921-01-01", "2099-12-31")

# 8 GCMs and 2 SSPs per GCM
gcm_list  = ["CMCC-ESM2", "GFDL-ESM4", "INM-CM5-0", "KACE-1-0-G", "MIROC6", "MPI-ESM1-2-HR", "MPI-ESM1-2-LR", "MRI-ESM2-0"] 
ssp_list  = ["ssp126", "ssp585"] 

chunks_dict = {"lon": 50, "lat": 50, "time": -1}
encode_t2m  = {'tas': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999}}
encode_pp   = {"pr":  {"zlib": True, "complevel": 1, "dtype": "float32"}}

In [ ]:
for gcm in tqdm(gcm_list):    
    for ssp in tqdm(ssp_list, leave = False):
        
        gcm_pp  = xr.open_mfdataset("future_corrected/PP_" + gcm + "_" + ssp + "*.nc", chunks = "auto").pr
        gcm_pp.attrs['units']   = "mm month-1"
        gcm_pp = gcm_pp.resample(time = "MS").sum()
        gcm_pp = gcm_pp.chunk("auto").rename("pr")
        gcm_pp = core.units.convert_units_to(gcm_pp, target = 'kg m-2 s-1', context = "hydro")
        
        gcm_t2m = xr.open_mfdataset("future_corrected/T2M_" + gcm + "_" + ssp + "*.nc", chunks = "auto").t2m
        gcm_t2m.attrs['units']   = "C"
        gcm_t2m = gcm_t2m.resample(time = "MS").mean()
        gcm_t2m = gcm_t2m.chunk("auto").rename("tas")
        gcm_t2m = core.units.convert_units_to(gcm_t2m, target = 'K')

        # save files
        gcm_pp.to_netcdf("/home/rooda/Hydro_results/future_glaciers/PP_" + gcm + "_" + ssp + ".nc", encoding = encode_pp)
        gcm_t2m.to_netcdf("/home/rooda/Hydro_results/future_glaciers/T2M_" + gcm + "_" + ssp + ".nc", encoding = encode_t2m)